In [ ]:
# dependencies and setup
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# Store CSV to DataFrame
listing_host_csv = "Resources/listings.csv"
listing_host_df = pd.read_csv(listing_host_csv,encoding="utf8")

In [ ]:
# change column "name" to something that SQL doesn't already recognize
listing_host_df.rename(index=str,columns={"name":"listing_name"},inplace=True)
listing_host_df.head()

# Creating the "Listings" dataframe and cleaning it

In [ ]:
# Create new data with select columns for the AirBnB listings
listing_df = listing_host_df[["id","listing_name","street","neighbourhood_cleansed","neighbourhood_group_cleansed","city","state","zipcode","latitude","longitude","is_location_exact","property_type","room_type","accommodates","bathrooms","bedrooms","beds","bed_type","square_feet","price","weekly_price","monthly_price","security_deposit","cleaning_fee","guests_included","extra_people","minimum_nights","maximum_nights","has_availability","availability_30","availability_60","availability_90","availability_365","number_of_reviews","first_review","last_review","review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value","requires_license","instant_bookable","cancellation_policy","require_guest_profile_picture","require_guest_phone_verification","reviews_per_month","host_id"]].copy()

In [ ]:
# convert dates to datetime
listing_df["first_review"]=pd.to_datetime(listing_df["first_review"])
listing_df["last_review"]=pd.to_datetime(listing_df["last_review"])

In [ ]:
# convert boolean columns to boolean
listing_df["is_location_exact"].replace(["t","f"],[True,False],inplace=True)
listing_df["has_availability"].replace(["t","f"],[True,False],inplace=True)
listing_df["requires_license"].replace(["t","f"],[True,False],inplace=True)
listing_df["instant_bookable"].replace(["t","f"],[True,False],inplace=True)
listing_df["require_guest_profile_picture"].replace(["t","f"],[True,False],inplace=True)
listing_df["require_guest_phone_verification"].replace(["t","f"],[True,False],inplace=True)

In [ ]:
# convert all the currency columns to numeric values instead of strings
# define the currency columns
currency_cols=["price","weekly_price","monthly_price","security_deposit","cleaning_fee","extra_people"]
# remove dollar sign and commas
listing_df[currency_cols]=listing_df[currency_cols].replace({'\$': '', ',': ''}, regex=True)
# changing the leftover value (minus currency sign and commas) to a number
listing_df["price"] = pd.to_numeric(listing_df["price"])
listing_df["weekly_price"] = pd.to_numeric(listing_df["weekly_price"])
listing_df["monthly_price"] = pd.to_numeric(listing_df["monthly_price"])
listing_df["security_deposit"] = pd.to_numeric(listing_df["security_deposit"])
listing_df["cleaning_fee"] = pd.to_numeric(listing_df["cleaning_fee"])
listing_df["extra_people"] = pd.to_numeric(listing_df["extra_people"])
listing_df

# Creating the "Hosts" dataframe and cleaning it

In [ ]:
# Create new data with select columns for the AirBnB hosts
host_df = listing_host_df[["host_id","host_name","host_since","host_location","host_response_time","host_response_rate","host_acceptance_rate","host_is_superhost","host_neighbourhood","host_listings_count","host_has_profile_pic","host_identity_verified"]].copy()

In [ ]:
# delete duplicates
host_df.drop_duplicates(keep="first",inplace=True)

In [ ]:
# convert dates to datetime format
host_df["host_since"]=pd.to_datetime(host_df["host_since"])

In [ ]:
# convert boolean columns to boolean
host_df["host_is_superhost"].replace(["t","f"],[True,False],inplace=True)
host_df["host_has_profile_pic"].replace(["t","f"],[True,False],inplace=True)
host_df["host_identity_verified"].replace(["t","f"],[True,False],inplace=True)

In [ ]:
# converting the percentage columns from strings to percentages
# remove the percentage symbols from the columns with percentages
host_df["host_response_rate"]=host_df["host_response_rate"].replace({'\$': '', ',': '', '%':''}, regex=True)
host_df["host_acceptance_rate"]=host_df["host_acceptance_rate"].replace({'\$': '', ',': '', '%':''}, regex=True)
# convert the percentage columns to numbers
host_df["host_response_rate"] = pd.to_numeric(host_df["host_response_rate"])
host_df["host_acceptance_rate"] = pd.to_numeric(host_df["host_acceptance_rate"])
host_df

# Creating the "Availability" dataframe and cleaning it¶

In [ ]:
# Store CSV to DataFrame
availability_file = "Resources/calendar.csv"
availability_df = pd.read_csv(availability_file,encoding="utf8")

In [ ]:
# change column "date" to something that SQL doesn't already recognize
availability_df.rename(index=str,columns={"date":"available_date"},inplace=True)
# convert column to datetime
availability_df["available_date"]=pd.to_datetime(availability_df["available_date"])

In [ ]:
# convert boolean columns to boolean
availability_df["available"].replace(["t","f"], [True,False], inplace=True)

In [ ]:
# remove dollar sign
availability_df["price"] = availability_df["price"].replace({'\$': '', ',': ''}, regex=True)
# convert the price column to numeric values instead of strings
availability_df["price"] = pd.to_numeric(availability_df["price"])
availability_df.head()

# Creating the "Reviews" dataframe and cleaning it

In [ ]:
# Store CSV to DataFrame
reviews_file = "Resources/reviews.csv"
reviews_df = pd.read_csv(reviews_file,encoding="utf8")

In [ ]:
# change column "date/id" to something that SQL doesn't already recognize
reviews_df.rename(index=str,columns={"date":"review_date"},inplace=True)
reviews_df.rename(columns={"id":"review_id"}, inplace=True)

In [ ]:
# Create new data with select columns for the AirBnB listings
review_df = reviews_df[['review_id', "listing_id", "review_date", "reviewer_id", "reviewer_name", "comments"]].copy()

In [ ]:
# convert dates to datetime
review_df["review_date"] = pd.to_datetime(review_df["review_date"])
review_df.head()

# SQL

In [ ]:
# establish the MySQL database connection and create the engine
rds_connection_string = "root:<INSERT PASSWORD>@127.0.0.1/seattle_airbnb_db"
engine = create_engine(f"mysql+pymysql://{rds_connection_string}?charset=utf8", encoding = "utf8")

In [ ]:
# check tablenames in the MySQL database
engine.table_names()

In [ ]:
# add the hosts dataframe to MySQL
host_df.to_sql(name="airbnb_hosts",con=engine,if_exists="append",index=False,chunksize=2000)

In [ ]:
# add the listings dataframe to MySQL
listing_df.to_sql(name="listings",con=engine,if_exists="append",index=False,chunksize=200)

In [ ]:
# add the property availability dataframe to MySQL
availability_df.to_sql(name="property_availability",con=engine,if_exists="append",index=False,chunksize=2000)

In [ ]:
# add the property reviews dataframe to MySQL
reviews_df.to_sql(name="property_reviews",con=engine,if_exists="append",index=False,chunksize=2000)